In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from easydict import EasyDict as edict

import sys
#sys.path.append('./src')

from src.dataset import get_loader
from src.cifar_trainer import Trainer
import src.distrib as distrib
from cifar_model.quant_resnet4cifar_my import resnet8_quant, resnet20_quant, QBasicBlock4Cifar
from cifar_model.quant_conv import QConv

from imagenet_model.quant_resnet_my import resnet18_quant

In [2]:
args =edict()
args.quant = edict()


args['db'] = edict()
args.db.name = 'imagenet'
args.db.root = '/dataset/imagenet'

args.quant['arch'] = 'resnet20_quant' if args.db.name == 'cifar10' else 'resnet18_quant'
args.quant['QWeightFlag'] = True
args.quant['QActFlag'] = True
args.quant['bkwd_scaling_factorW'] = 1.0
args.quant['bkwd_scaling_factorA'] = 1.0
args.quant['groups']=4
args.quant.bit_list = ['2','4','6','8']

args.lr_sched = None
args.device = 'cuda:0'
args.epochs= 200
args.optim= 'sgd'
args.lr= 0.1
args.momentum= 0.9
args.w_decay= 5e-4
args.batch_size= 128
args.mixed= True  # if true, uses mixed precision training
args.beta2= 0.999
args.max_norm= 5
args.nesterov= True
args.alpha= 0.9
args.continue_from=False
args.checkpoint=False
args.history_file=None
args.pre_load_pretrained = True
args.restart= False # Ignore existing checkpoints
args.checkpoint_file= 'checkpoint.th'
args.history_file= 'history.json'
args.num_prints= 10

In [3]:
img_size = 32 if 'cifar' in args.db.name else 224 
trainset, testset, num_classes = get_loader(args, img_size)
criterion = nn.CrossEntropyLoss()
data = edict()
data['tr'] = distrib.loader(trainset, args.batch_size, shuffle=True, num_workers=4)
data['tt'] = distrib.loader(testset, args.batch_size, shuffle=False, num_workers=4)

In [4]:
if args.quant.arch == 'resnet20_quant':
    model = resnet20_quant(args.quant)
else:
    model = resnet18_quant(args.quant)
    checkpoint_path = './r18-2468/checkpoint.pth.tar'
    checkpoint = torch.load(checkpoint_path)
    woddp_checkpoint = {}
    for key, value in checkpoint['state_dict'].items():
        woddp_checkpoint[key.replace('module.','')] = value
    model.load_state_dict(woddp_checkpoint, strict=False)
model.cuda()

QResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): QBasicBlock(
      (conv1): QConv(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False
        (uW): ParameterList(
            (0): Parameter containing: [torch.float32 of size  (cuda:0)]
            (1): Parameter containing: [torch.float32 of size  (cuda:0)]
            (2): Parameter containing: [torch.float32 of size  (cuda:0)]
            (3): Parameter containing: [torch.float32 of size  (cuda:0)]
        )
        (lW): ParameterList(
            (0): Parameter containing: [torch.float32 of size  (cuda:0)]
            (1): Parameter containing: [torch.float32 of size  (cuda:0)]
            (2): Parameter contai

In [5]:
trainer = Trainer(data, model, criterion, None, args)

In [7]:
trainer.evaluate()

91014.16366577148
50000
439499.8070983887
50000
402021.19038391113
50000
74216.29936599731
50000
{'2_loss': [1.82028329372406], '2_Acc1': [56.67599868774414], '2_Acc5': [80.90599822998047], '4_loss': [8.789996147155762], '4_Acc1': [0.06199999898672104], '4_Acc5': [0.41999998688697815], '6_loss': [8.040423393249512], '6_Acc1': [0.12800000607967377], '6_Acc5': [0.5299999713897705], '8_loss': [1.484325885772705], '8_Acc1': [63.86800003051758], '8_Acc5': [85.72200012207031]}


ValueError: too many values to unpack (expected 2)

In [ ]:
abc = 63590.97902679443
d = 10000

if not isinstance(abc, torch.Tensor):
    abc = torch.tensor(abc)

tensor = torch.tensor([abc] + [1])
tensor *= d
print(tensor)
tensor 

tensor([6.3591e+08, 1.0000e+04])
